# Dynamic Programming - Knapsack Problem

经典的 0-1 背包问题

有 n 种物品，每种物品有固定的价值 v，和重量 w，另外有一个可以承载有限重量的背包，每种物品可以选择装或不装，问如何装包使的背包内物品总价值最高。


解题的关键还是理解“最优决策序列”这件事，并且定义好状态转移方程

$$
dp(i, w) = max\{dp(i - 1, w), dp(i - 1, w - w_i) + v_i\}
$$

其中 i 为第 i 个物品，w 为限制的重量，函数dp的含义为可以放前 i 个物品，并限制重量为 w 时，能够获取的最大价值。
$w_i$为第 i 个物品的重量，$v_i$为第 i个物品的价值

网上大部分介绍的方法都是构造 dp 矩阵进行求解，然后从dp(0, w)一路往 dp(n, weight_limit)推演，不过我觉得这种方法可能存在以下几个问题：

- 使用dp矩阵正推当 weight_limit 非常大时，构造出来的矩阵维数会非常大
- 使用dp矩阵正推当物品重量为小数时，要换成整数才行

此外，网上大部分资料都没有提到，如何计算出背包里究竟要放那些东西

In [101]:
# question setup

class Item():
    def __init__(self, name, value, weight):
        self.name = name
        self.value = value
        self.weight = weight
    def __repr__(self):
        return "{} (v: {}, w: {})".format(self.name, self.value, self.weight)
        
items = [
    Item("laptop", 10, 12),
    Item("glass", 6, 3),
    Item("t-shirt", 7, 9),
    Item("umbrella", 20, 7),
    Item("phone", 4, 9),
]

weight_limit = 15

In [104]:
# define solver

import numpy as np

# 用 Hash 记忆加递归求解的一个好处是可以直接处理物品重量为小数的情况
def solve_recursive(items, weight_limit):
    
    def dp(i, w, items, mem):
        mem_key = i * (weight_limit + 1) + w
        if mem_key in mem:       # already calculated
            return mem[mem_key]
        elif i == 0:             # first item (boudary)
            if w >= items[i].weight:
                val = items[i].value
            else:
                val = 0
            mem[mem_key] = val
            return val        
        else:
            if w >= items[i].weight:
                val = max(dp(i - 1, w, items, mem), dp(i - 1, w - items[i].weight, items, mem) + items[i].value)
            else:
                val = dp(i - 1, w, items, mem)
            mem[mem_key] = val
            return val
    
    mem = {}
    max_val = dp(len(items) - 1, weight_limit, items, mem)
    return max_val


def solve_dp_matrix(items, weight_limit):
    
    dp = np.zeros((len(items), weight_limit + 1), "int")
    
    for i in range(len(items)):
        for w in range(weight_limit + 1):
            if i == 0:
                dp[i][w] = items[i].value if items[i].weight <= w else 0
            else:
                if w >= items[i].weight:
                    dp[i][w] = max(
                        dp[i - 1][w],
                        dp[i - 1][w - items[i].weight] + items[i].value
                    )
                else:
                    dp[i][w] = dp[i - 1][w]
    
    return dp[-1, -1]


def solve_dp_scroll_array(items, weight_limit):
    
    dp = np.zeros(weight_limit + 1, 'int')
    
    for i in range(len(items)):
        for w in range(weight_limit, -1, -1):
            if w >= items[i].weight:
                dp[w] = max(
                    dp[w],
                    dp[w - items[i].weight] + items[i].value
                )
    
    return dp[-1]


ret = solve_recursive(items, weight_limit)
print("[Recursive] Max value is: ", ret)

ret = solve_dp_matrix(items, weight_limit)
print("[Dp Matrix] Max value is: ", ret)

ret = solve_dp_scroll_array(items, weight_limit)
print("[Dp Scroll Array] Max value is: ", ret)

[Recursive] Max value is:  26
[Dp Matrix] Max value is:  26
[Dp Scroll Array] Max value is:  26


In [105]:
# 使用递归求解，并计算出具体需要装包的物品清单
def solve_recursive_with_pack_list(items, weight_limit):
    
    def mem_key(i, w):
        return i * (weight_limit + 1) + w
    
    def dp(i, w, items, mem):
        key = mem_key(i, w)
        if key in mem:       # already calculated
            return mem[key]["value"]
        elif i == 0:             # first item (boudary)
            if w >= items[i].weight:
                val = items[i].value
                take = True
            else:
                val = 0
                take = False
            mem[key] = {
                "value": val,
                "item": items[i] if take else None,
                "prev_key": None,
            }
            return val    
        else:
            if w >= items[i].weight:
                v1 = dp(i - 1, w, items, mem)
                v2 = dp(i - 1, w - items[i].weight, items, mem) + items[i].value
                if v1 >= v2:
                    val = v1
                    take = False
                    prev_key = mem_key(i - 1, w)
                else:
                    val = v2
                    take = True
                    prev_key = mem_key(i - 1, w - items[i].weight)
            else:
                val = dp(i - 1, w, items, mem)
                take = False
                prev_key = mem_key(i - 1, w)
                
            mem[key] = {
                "value": val,
                "item": items[i] if take else None,
                "prev_key": prev_key
            }
            return val
    
    mem = {}
    max_val = dp(len(items) - 1, weight_limit, items, mem)
    
    # back trace decision tree from mem
    pack_list = []
    k = mem_key(len(items) - 1, weight_limit)
    print("key: ", k)
    while k != None:
        if mem[k]["item"]:
            pack_list.append(mem[k]["item"])
        k = mem[k]["prev_key"]
    
    print(pack_list)
    return max_val, pack_list

ret = solve_recursive_with_pack_list(items, weight_limit)
print("========== [Recursive With Pack List] ===========")
print("max value is ", ret[0])
print("----------- packing list ----------")
for i in ret[1]:
    print(i)

key:  79
[umbrella (v: 20, w: 7), glass (v: 6, w: 3)]
========== [Recursive With Pack List] ===========
max value is  26
----------- packing list ----------
umbrella (v: 20, w: 7)
glass (v: 6, w: 3)
